In [1]:
import json, random, re, collections, itertools, base64, sys
from dataclasses import dataclass
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(x))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
annot_path = Path("../annotations")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

%load_ext autoreload
%autoreload 2

## Read mapping

In [2]:
with (preprocessed_path / "ingredients.json").open("r") as f:
    ingredients = json.load(f)
with (preprocessed_path / "labels.json").open("r") as f:
    idx2label = [tuple(t) for t in json.load(f)]
    label2idx = {r: i for i, r in enumerate(idx2label)}
with (preprocessed_path / "resources.json").open("r") as f:
    resources = json.load(f)
ingredient_dict = {k: v for k, v in ingredients}
resource_dict = {k: v for k, v in resources}

with (data_path/"annotaions.json").open('r') as f:
    annotations = json.load(f)

In [3]:
ls(preprocessed_path)

['../preprocessed/103308.npz',
 '../preprocessed/ingredients.json',
 '../preprocessed/resources.json',
 '../preprocessed/labels.json',
 '../preprocessed/103308.json']

In [4]:
states = np.load(str(preprocessed_path/ "103308.npz" ))
states.files

['103308_0',
 '103308_1',
 '103308_2',
 '103308_3',
 '103308_4',
 '103308_5',
 '103308_6',
 '103308_7',
 '103308_8',
 '103308_9']

In [5]:
states['103308_3']

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [6]:
annotations["103308"]

[{'0': [['A1', 'I10_d4pRP'],
   ['A1', 'I7oUHHY41'],
   ['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'IukXkN8DV'],
   ['A1', 'Iynxt1P5R'],
   ['A1', 'LIMMEDIATE']]},
 {'1': [['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'IukXkN8DV'],
   ['A1', 'Iynxt1P5R'],
   ['A1', 'LIMMEDIATE'],
   ['GW4', 'I10_d4pRP'],
   ['GW4', 'I7oUHHY41'],
   ['GW4', 'LIMMEDIATE'],
   ['GW4', 'TSKILLET']],
  '0': [['A1', 'I10_d4pRP'],
   ['A1', 'I7oUHHY41'],
   ['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'IukXkN8DV'],
   ['A1', 'Iynxt1P5R'],
   ['A1', 'LIMMEDIATE']]},
 {'1': [['A1', 'IN0CxTNVh'],
   ['A1', 'INNbMITPe'],
   ['A1', 'IW2FkWnJk'],
   ['A1', 'Itg64O_Uz'],
   ['A1', 'LIMMEDIATE'],
   ['GW4', 'I10_d4pRP'],
   ['GW4', 'I7oUHHY41'],
   ['GW4', 'LIMMEDIATE'],
   ['GW4', 'TSKILLET'],
   ['W1', 'IukXkN8DV'],
   ['W1', 'Iynxt1P5R'],
   

In [7]:
from instruction_parsing import program, program_step, Instruction

def human_program(annotation):
    """Runs the program and translates to human language"""
    actions = [
        (
            a.ts,
            a.command.name,
            ingredient_dict.get(a.ingredient, resource_dict.get(a.ingredient, "")),
            resource_dict.get(a.resource),
        )
        for a in program(annotation)
    ]
    return actions

pp (human_program(annotations["103308"]))

[(1, 'USE', 'Skillet', 'Stove/Medium-High'),
 (1, 'PUT', 'vegetable oil', 'Stove/Medium-High'),
 (1, 'PUT', 'butter', 'Stove/Medium-High'),
 (2, 'PUT', 'boneless pork chops', 'Countertop/Combine/Mix'),
 (2, 'PUT', 'salt and ground black pepper to taste', 'Countertop/Combine/Mix'),
 (3, 'MOVE_CONTENTS', 'Countertop/Combine/Mix', 'Countertop/Coat/Sprinkle'),
 (3, 'PUT', 'all-purpose flour', 'Countertop/Coat/Sprinkle'),
 (4, 'REMOVE', 'all-purpose flour', 'Countertop/Coat/Sprinkle'),
 (4,
  'REMOVE',
  'salt and ground black pepper to taste',
  'Countertop/Coat/Sprinkle'),
 (4, 'REMOVE', 'boneless pork chops', 'Countertop/Coat/Sprinkle'),
 (4, 'PUT', 'all-purpose flour', 'Stove/Medium-High'),
 (4, 'PUT', 'salt and ground black pepper to taste', 'Stove/Medium-High'),
 (4, 'PUT', 'boneless pork chops', 'Stove/Medium-High'),
 (4, 'CHEF_CHECK', 'For X minutes', 'Stove/Medium-High'),
 (5, 'REMOVE', 'salt and ground black pepper to taste', 'Stove/Medium-High'),
 (5, 'REMOVE', 'boneless pork cho

In [8]:
program(annotations["103308"])

[Instruction(ts=1, command=<Commands.USE: 2>, ingredient='TSKILLET', resource='GW4'),
 Instruction(ts=1, command=<Commands.PUT: 0>, ingredient='I7oUHHY41', resource='GW4'),
 Instruction(ts=1, command=<Commands.PUT: 0>, ingredient='I10_d4pRP', resource='GW4'),
 Instruction(ts=2, command=<Commands.PUT: 0>, ingredient='Iynxt1P5R', resource='W1'),
 Instruction(ts=2, command=<Commands.PUT: 0>, ingredient='IukXkN8DV', resource='W1'),
 Instruction(ts=3, command=<Commands.MOVE_CONTENTS: 5>, ingredient='W1', resource='W5'),
 Instruction(ts=3, command=<Commands.PUT: 0>, ingredient='IW2FkWnJk', resource='W5'),
 Instruction(ts=4, command=<Commands.REMOVE: 1>, ingredient='IW2FkWnJk', resource='W5'),
 Instruction(ts=4, command=<Commands.REMOVE: 1>, ingredient='IukXkN8DV', resource='W5'),
 Instruction(ts=4, command=<Commands.REMOVE: 1>, ingredient='Iynxt1P5R', resource='W5'),
 Instruction(ts=4, command=<Commands.PUT: 0>, ingredient='IW2FkWnJk', resource='GW4'),
 Instruction(ts=4, command=<Commands.PU

In [9]:
for a,b in zip(annotations["103308"], annotations["103308"][1:]):
    last = str(max(map(int,a.keys())))
    last = set(filter(lambda t: t[1].startswith('I'),map(tuple, a[last])))
    first = set(filter(lambda t: t[1].startswith('I'),map(tuple, b['0'])))
    print (last-first)

set()
set()
set()
set()
set()
set()
set()
set()
set()


In [10]:
program_step(annotations["103308"][1])

[Instruction(ts=1, command=<Commands.USE: 2>, ingredient='TSKILLET', resource='GW4'),
 Instruction(ts=1, command=<Commands.PUT: 0>, ingredient='I7oUHHY41', resource='GW4'),
 Instruction(ts=1, command=<Commands.PUT: 0>, ingredient='I10_d4pRP', resource='GW4')]

In [11]:
annotations["103308"][1]

{'1': [['A1', 'IN0CxTNVh'],
  ['A1', 'INNbMITPe'],
  ['A1', 'IW2FkWnJk'],
  ['A1', 'Itg64O_Uz'],
  ['A1', 'IukXkN8DV'],
  ['A1', 'Iynxt1P5R'],
  ['A1', 'LIMMEDIATE'],
  ['GW4', 'I10_d4pRP'],
  ['GW4', 'I7oUHHY41'],
  ['GW4', 'LIMMEDIATE'],
  ['GW4', 'TSKILLET']],
 '0': [['A1', 'I10_d4pRP'],
  ['A1', 'I7oUHHY41'],
  ['A1', 'IN0CxTNVh'],
  ['A1', 'INNbMITPe'],
  ['A1', 'IW2FkWnJk'],
  ['A1', 'Itg64O_Uz'],
  ['A1', 'IukXkN8DV'],
  ['A1', 'Iynxt1P5R'],
  ['A1', 'LIMMEDIATE']]}